<a href="https://colab.research.google.com/github/dlsun/Stat350F19/blob/master/Filter%20Design%20in%20Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#@title Import Plotting
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
#@title Import Audio
from IPython.display import Audio
from scipy.io import wavfile
from io import BytesIO

## Generating a Signal

First, let's generate a signal with frequencies 256 Hz and 554 Hz in equal parts.

In [0]:
fs = 8000
t = np.arange(0, 5, step=1. / fs)
x = np.sin(2 * np.pi * 256 * t) + np.sin(2 * np.pi * 554 * t)

plt.plot(t, x)
plt.xlim(0, .1)

You can use the `Audio()` function inside `IPython.display` to listen to any signal. Don't forget to specify the sampling rate.

In [0]:
Audio(x, rate=fs)

## Frequency Analysis of the Signal

`np.fft.fft` is an implementation of the Fast Fourier Transform (FFT), which is an efficient algorithm for calculating the Discrete Fourier Transform (DFT).

If you pass in a time-domain signal of length $N$ to `np.fft.fft`, you will get back a frequency-domain signal of length $N$. 
- The first $\frac{N}{2}$ samples in the frequency domain represent the DFT values at equally spaced _positive_ frequencies in $[0, f_s / 2)$.
- The last $\frac{N}{2}$ samples represent the DFT values at equally spaced _negative_ frequencies in $[-f_s / 2, 0)$.

If the time-domain signal is real-valued, then the DFT values at negative frequencies will be the complex conjugates of the DFT values at positive frequencies. So all of the information is contained in the first half of the DFT for a real-valued signal.

In [0]:
X = np.fft.fft(x)
X

Since the values of the DFT are complex numbers, we often examine the magnitude or the power (magnitude squared) of these numbers.

In [0]:
X_pow = np.abs(X) ** 2
plt.plot(X_pow)

So far, we have only plotted the values of the DFT. Let's recreate this plot with the correct frequencies on the $x$-axis by defining `f_pos` to be an array of $N / 2$ equally spaced frequencies between $0$ and $f_s / 2$.

In [0]:
N = len(x)
f_pos = np.arange(0, fs / 2, step=fs / N)
plt.plot(f_pos, X_pow[:(N // 2)])

N, len(f_pos)

Now let's add the negative frequencies as well.

In [0]:
N = len(x)

# Plot the positive frequencies.
f_pos = np.arange(0, fs / 2, step=fs / N)
plt.plot(f_pos, X_pow[:(N // 2)])

# Plot the negative frequencies.
f_neg = np.arange(-fs / 2, 0, step=fs / N)
plt.plot(f_neg, X_pow[(N // 2):])

# Now we can label the x-axis.
plt.xlabel("Frequency (Hz)")

## Designing the Filter

Let's design a filter that keeps the 256 Hz sinusoid, but removes the 554 Hz sinusoid.

We first design the filter that we want in the frequency domain. A lowpass filter with a cutoff frequency of 400 Hz should do the trick.

First, define the transfer function / frequency response $H(f)$ at the positive frequencies $f \geq 0$.

In [0]:
f_pos = np.arange(0, fs / 2, step=fs / N)
H_pos = 1. * (f_pos <= 400)
plt.plot(f_pos, H_pos)
plt.ylim(-.1, 1.1)

Now, define the transfer function / frequency response $H(f)$ at the negative frequencies $f < 0$.

In [0]:
f_neg = np.arange(-fs / 2, 0, step=fs / N)
H_neg = 1. * (f_neg >= -400)
plt.plot(f_neg, H_neg)
plt.ylim(-.1, 1.1)

Finally, let's combine the frequency response at the positive and negative frequencies. Remember that the negative frequencies go at the _end_, after the positive frequencies.

In [0]:
H = np.concatenate([H_pos, H_neg])
plt.plot(H)

Now, we need to convert this frequency-domain filter to the time-domain. To do this, we take the _inverse_ FFT to obtain the impulse response of this filter.

In [0]:
h = np.real(np.fft.ifft(H)) # The IFFT should automatically be real-valued. We take np.real just to be safe.
plt.plot(h)

Let's truncate this impulse response to the first 1000 samples (i.e., positive time) and last 1000 samples (i.e., negative time). We also arrange this impulse response in the right order, with the negative times coming first.

In [0]:
h_trunc = np.concatenate([h[-1000:], h[:1000]])
plt.plot(h_trunc)

## Applying the Filter

Now we apply the filter by convolving the impulse response with the signal.

In [0]:
y = np.convolve(x, h_trunc)

# y has an odd number of samples, so we lop off the last one.
y = y[:-1]

In [0]:
N = len(y)
t = np.arange(0, N / fs, step=1 / fs)
plt.plot(t, y)
plt.xlim(.3, .4)

Let's do a frequency analysis of this signal.

In [0]:
f_pos = np.arange(0, fs / 2, step=fs / N)
Y = np.fft.fft(y)
Y_pow = np.abs(Y) ** 2

plt.plot(f_pos, Y_pow[:(N // 2)])

Finally, let's listen to the filtered signal.

In [0]:
Audio(y, rate=fs)